In [13]:
import time,datetime, threading
from flirpy.camera.boson import Boson
import numpy as np
import gpsd
import os
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

from matplotlib import pyplot as plt
%matplotlib inline 

WAIT_SECONDS=1

In [2]:
#start the GPS daemon
cmd="sudo gpsd /dev/serial0 -F /var/run/gpsd.sock"
val=os.system(cmd)

In [6]:
#setup the output directory
basedir="/home/pi/thermalcamera/data/"+time.strftime("/%Y/%m/%d")

#Determine the run number up to 100
i=1
while i <= 100:
    dir=basedir+"/run"+str(i).zfill(3)
    if os.path.isdir(dir) is False:
        os.makedirs(dir)
        break
    else:
        i+=1

print(dir)

#connect the GPS
gpsd.connect()

/home/pi/thermalcamera/data//2020/10/05/run001


In [ ]:
#setup the output structures
anc={}
gps={}

#determine if there is a Boson present or not
i=0
if Boson.find_video_device() is None:
    print("ERROR: No Boson Found")
else:
    with Boson() as camera:
        
        #Make the Boson do FFCs in a given pattern so we can make sure its actually talking
        for x in range(1,10):
            if x%4 == 0:
                print("ffc"+str(x))
                #camera.do_ffc()
            if x%5 ==0:
                print("ffc"+str(x))
                #camera.do_ffc()
            time.sleep(1)
            
        while True:
            i+=1
            tstart = time.time()
            img = camera.grab()
            assert img.dtype == "uint16"
    
            #do an FFC on every 60th image
            if i%60 == 0:
                camera.do_ffc()
            
            #get the ancillary stuff from the Boson
            anc['serial']=camera.get_sensor_serial()
            anc['firmware']=camera.get_firmware_revision()
            anc['ffc_temp']=camera.get_last_ffc_temperature()
            anc['fpa_temp']=camera.get_fpa_temperature()
            anc['frame_count']=camera.get_frame_count()
            anc['ffc_frame_count']=camera.get_last_ffc_frame_count()
            anc['ffc_mode']=camera.get_ffc_mode()
            anc['seq_count']=i
        
            #get the GPS packet but only if it has a fix
            packet = gpsd.get_current()
            if packet.mode == 3:
                gps['fix_type']=packet.mode()
                gps['altitude']=packet.altitude()
                gps['time']=packet.get_time()
                gps['hspeed']=packet.hspeed()
                gps['vspeed']=packet.speed_vertical()
                gps['sats_valid']=packet.sats_valid()
                gps['sats']=packet.sats()
                position=packet.position()
                gps['position_precision']=packet.position_precision()
                gps['lon']=position[0]
                gps['lat']=position[1]
            
            print(i)
            #print(anc)
            #print(gps)
            #plt.imshow(img)
            #plt.show()
            
            out = Image.fromarray(img) # float32
            out.save(dir+"/img_"+str(i).zfill(5)+".tiff", "TIFF",compression="tiff_deflate")
            
            #sleep for 1 second between images
            time.sleep(0.75) 
            
            tend = time.time()
            print(tend-tstart)

ffc4
ffc5
ffc8
1
1.1443378925323486
2
0.9269790649414062
3
0.9200952053070068
4
0.9151053428649902
5
0.9049797058105469
6
0.9206602573394775
7
0.9163234233856201
8
0.9153563976287842
9
0.9483733177185059
10
0.9425928592681885
11
0.9261658191680908
12
0.9308156967163086
13
0.9138705730438232
14
0.9204905033111572
15
0.9149866104125977
16
0.9439835548400879
17
0.9364702701568604
18
0.9993586540222168
19
1.0333623886108398
20
0.9531807899475098
21
0.8861491680145264
22
0.9442358016967773
23
0.9705240726470947
24
0.9448623657226562
25
0.9475128650665283
26
0.9638760089874268
27
0.9962892532348633
28
0.9401512145996094
29
0.9132301807403564
30
0.909724235534668
31
0.9422547817230225
32
0.9733362197875977
33
0.9754021167755127
34
0.9578604698181152
35
0.9865233898162842
36
0.9469804763793945
37
0.9608023166656494
38
0.9546246528625488
39
0.9676754474639893
40
0.945488452911377
41
0.9065368175506592
42
0.9669253826141357
43
0.9042916297912598
44
0.941992998123169
45
0.9012367725372314
46
0.94

go


<Timer(Thread-6, initial)>